In [1]:
import jax

import jax.numpy as np
import jax.scipy as sp

from numpy.polynomial.hermite import hermgauss
import scipy as osp

from vb_lib import structure_model_lib, data_utils
import vb_lib.structure_optimization_lib as s_optim_lib
import vb_lib.structure_preconditioned_optimization_lib as s_poptim_lib

import paragami

from copy import deepcopy

import time

import matplotlib.pyplot as plt
%matplotlib inline  

from bnpmodeling_runjingdev import cluster_quantities_lib, modeling_lib
from bnpmodeling_runjingdev.optimization_lib import run_lbfgs


/accounts/grad/runjing_liu/.conda/envs/bnp_sensitivity_jax/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import numpy as onp
onp.random.seed(53453)

# Load data

In [3]:
# data = np.load('../../simulated_data/simulated_structure_data_nobs20_nloci50_npop4.npz')
# g_obs = np.array(data['g_obs'], dtype = int)

In [4]:
data_dir = '../../../../../fastStructure/hgdp_data/huang2011_plink_files/'
filenamebase = 'phased_HGDP+India+Africa_2810SNPs-regions1to36'
filename = data_dir + filenamebase + '.npz'
data = np.load(filename)

g_obs = np.array(data['g_obs'], dtype = int)
g_obs_raw = np.array(data['g_obs_raw'])

# just checking ... 
which_missing = (g_obs_raw == 3)
(g_obs.argmax(-1) == g_obs_raw)[~which_missing].all()
(g_obs[which_missing] == 0).all()

In [5]:
n_obs = g_obs.shape[0]
n_loci = g_obs.shape[1]

In [6]:
print(n_obs)
print(n_loci)

20
50


# Get prior

In [7]:
_, prior_params_paragami = \
    structure_model_lib.get_default_prior_params()

prior_params_dict = prior_params_paragami.random(key=jax.random.PRNGKey(41))

print(prior_params_paragami)

prior_params_free = prior_params_paragami.flatten(prior_params_dict, free = True)

OrderedDict:
	[dp_prior_alpha] = NumericArrayPattern (1,) (lb=0.0, ub=inf)
	[allele_prior_alpha] = NumericArrayPattern (1,) (lb=0.0, ub=inf)
	[allele_prior_beta] = NumericArrayPattern (1,) (lb=0.0, ub=inf)


# Get VB params 

In [8]:
k_approx = 8

In [9]:
gh_deg = 8
gh_loc, gh_weights = hermgauss(8)

In [10]:
use_logitnormal_sticks = True

vb_params_dict, vb_params_paragami = \
    structure_model_lib.get_vb_params_paragami_object(n_obs, n_loci, k_approx,
                                    use_logitnormal_sticks = use_logitnormal_sticks)
    
print(vb_params_paragami)

OrderedDict:
	[pop_freq_beta_params] = NumericArrayPattern (50, 8, 2) (lb=0.0, ub=inf)
	[ind_admix_params] = OrderedDict:
	[stick_means] = NumericArrayPattern (20, 7) (lb=-inf, ub=inf)
	[stick_infos] = NumericArrayPattern (20, 7) (lb=0.0001, ub=inf)


In [11]:
vb_params_dict = vb_params_paragami.random(key=jax.random.PRNGKey(41))

In [12]:
kl = structure_model_lib.get_kl(g_obs, 
                    vb_params_dict,
                    prior_params_dict, 
                    gh_loc, gh_weights)

print(kl)

2416.4788397129596


# Define preconditioned objective

In [13]:
# not preconditioned
objective = s_poptim_lib.StructurePrecondObjective(g_obs, 
                            vb_params_paragami,
                            prior_params_dict,
                            gh_loc = gh_loc, gh_weights = gh_weights, 
                            use_preconditioning = False)

compiling preconditioned objective ... 
done. Elasped: 14.5878


In [14]:
# preconditioned 
objective_precond = s_poptim_lib.StructurePrecondObjective(g_obs, 
                            vb_params_paragami,
                            prior_params_dict,
                            gh_loc = gh_loc, gh_weights = gh_weights)

compiling preconditioned objective ... 
done. Elasped: 19.8476


In [15]:
x = vb_params_paragami.flatten(vb_params_dict, free = True)
precond_params = x

x_c = objective_precond.precondition(x, x)

In [16]:
objective_precond.f_precond(x_c, precond_params) - kl

DeviceArray(0., dtype=float64)

In [17]:
objective.f_precond(x, None) - kl

DeviceArray(0., dtype=float64)

### Timing

In [25]:
for i in range(5): 
    t0 = time.time() 
    _ = objective_precond.f_precond(x_c, precond_params).block_until_ready()
    print(time.time() - t0)

0.0074920654296875
0.00696563720703125
0.007086038589477539
0.0073642730712890625
0.007078647613525391


In [27]:
for i in range(5): 
    t0 = time.time() 
    _ = objective.f_precond(x, None).block_until_ready()
    print(time.time() - t0)

0.0045108795166015625
0.003465414047241211
0.0034608840942382812
0.003240823745727539
0.003249645233154297


In [30]:
for i in range(5): 
    t0 = time.time() 
    _ = objective_precond.grad(x_c, precond_params).block_until_ready()
    print(time.time() - t0)

0.020404577255249023
0.017294883728027344
0.017186880111694336
0.016966819763183594
0.016858816146850586


In [31]:
for i in range(5): 
    t0 = time.time() 
    _ = objective.grad(x, None).block_until_ready()
    print(time.time() - t0)

0.013346433639526367
0.008729696273803711
0.008775949478149414
0.008796453475952148
0.008523702621459961
